In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/Users/neelbakshi/Documents/Machine Learning/Datasets/MNIST", one_hot = True)

Extracting /Users/neelbakshi/Documents/Machine Learning/Datasets/MNIST/train-images-idx3-ubyte.gz
Extracting /Users/neelbakshi/Documents/Machine Learning/Datasets/MNIST/train-labels-idx1-ubyte.gz
Extracting /Users/neelbakshi/Documents/Machine Learning/Datasets/MNIST/t10k-images-idx3-ubyte.gz
Extracting /Users/neelbakshi/Documents/Machine Learning/Datasets/MNIST/t10k-labels-idx1-ubyte.gz


In [61]:
class Network:
    
    def __init__(self, structure, input_placeholder, output_placeholder):
        self.structure = structure
        self.input_placeholder = input_placeholder
        self.output_placeholder = output_placeholder
        layers = []
        for current_nodes, previous_nodes in zip(self.structure[1:], self.structure[:-1]):
            weights = tf.Variable(tf.random_normal([previous_nodes, current_nodes]))
            biases = tf.Variable(tf.random_normal([current_nodes]))
            layers.append({'weights': weights, 'biases': biases})
            
        layer_output = self.input_placeholder
        for layer in layers[:-1]:
            layer_output = tf.add(tf.matmul(layer_output, layer['weights']), layer['biases'])
            layer_output = tf.nn.relu(layer_output)
        self.final_output = tf.add(tf.matmul(layer_output, layers[-1]['weights']), layers[-1]['biases'])
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.output_placeholder, logits=self.final_output))
        self.optimizer = tf.train.AdamOptimizer().minimize(self.cost)
        
    def fit(self, input_data, epochs, batch_size):
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for epoch in xrange(epochs):
                epoch_loss = 0
                for _ in range(int(input_data.train.num_examples/batch_size)):
                    epoch_x, epoch_y = input_data.train.next_batch(batch_size)
                    c, _ = sess.run([self.cost, self.optimizer], feed_dict={self.input_placeholder:epoch_x, self.output_placeholder:epoch_y})
                    epoch_loss += c
                print('Epoch', epoch, 'completed out of',epochs,'loss:',epoch_loss)
            
            correct = tf.equal(tf.argmax(self.final_output, 1), tf.argmax(self.output_placeholder, 1))

            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            print('Accuracy:',accuracy.eval({self.input_placeholder:input_data.test.images, self.output_placeholder:input_data.test.labels}))
                
        
    

In [62]:
network = Network([784, 500, 300, 200, 10], tf.placeholder('float', [None, 784]), tf.placeholder('float'))

In [63]:
network.fit(mnist, 10, 200)

('Epoch', 0, 'completed out of', 10, 'loss:', 638600.17395019531)
('Epoch', 1, 'completed out of', 10, 'loss:', 161408.22003173828)
('Epoch', 2, 'completed out of', 10, 'loss:', 100201.6109085083)
('Epoch', 3, 'completed out of', 10, 'loss:', 68701.049884796143)
('Epoch', 4, 'completed out of', 10, 'loss:', 48650.026763916016)
('Epoch', 5, 'completed out of', 10, 'loss:', 35658.974186897278)
('Epoch', 6, 'completed out of', 10, 'loss:', 25155.305115699768)
('Epoch', 7, 'completed out of', 10, 'loss:', 18822.050055503845)
('Epoch', 8, 'completed out of', 10, 'loss:', 13038.374998092651)
('Epoch', 9, 'completed out of', 10, 'loss:', 8977.2361810058355)
('Accuracy:', 0.93300003)
